# Memory-conservant size-limited file read
This is something I need from my job. I work on a test framework that runs test cases on our product, and that creates log messages. Sometimes we end up with lots of log data, enough to start filling up disk space and potentially crashing our test workers. So I want a queue-like thing with a file-like write method that happily accepts N bytes and passes them along for logging, and then anything after that, we just drop it on the floor.

In [30]:
import os
import tempfile


def noisy_source(output):
    # something like subprocess.Popen(...cmd..., output=output)
    assert output is not None
    for _ in range(5):
        output.write("abcde\n")


def get_tempfile_name():
    f = tempfile.NamedTemporaryFile()
    name = f.name
    f.close()
    return name


class LimitedPassthrough(object):
    def __init__(self, fn, limit):
        assert isinstance(fn, str), fn
        assert isinstance(limit, int), limit
        self.fn = fn
        self.limit = limit
        self._fileno = None
    def __enter__(self):
        self.f = f = open(self.fn, 'wb')
        self._fileno = f.fileno()
        return self
    def fileno(self):
        return self._fileno
    def write(self, x):
        self.f.write(f"{self.limit}\n".encode("utf-8"))
        x = x[:self.limit]
        self.limit -= len(x)
        self.f.write(x.encode("utf-8"))
    def __exit__(self, e1, e2, e3):
        assert e1 is None, (e1, e2, e3)
        # close after writing then reopen for reaading
        self.f.close()
        self.f = open(self.fn)
    def read(self):
        return self.f.read()
    def delete(self):
        try:
            self.f.close()
        except:
            pass
        os.unlink(self.fn)
        

SIX_BYTES = "abcde\n"
ACCEPT_JUST_A_FEW_BYTES = 8
name = get_tempfile_name()
with LimitedPassthrough(name, ACCEPT_JUST_A_FEW_BYTES) as output:
    assert output is not None
    for _ in range(5):
        output.write(SIX_BYTES)
print(output.read())
output.delete()

8
abcde
2
ab0
0
0



This looks pretty good so far but let's make it closer to the actual use case I really need.

In [32]:
import subprocess

cmd = ['python3', 'helper.py', '5']

with LimitedPassthrough(name, ACCEPT_JUST_A_FEW_BYTES) as output:
    assert output is not None
    p = subprocess.Popen(
        cmd,
        stdout=output,
        stderr=output
    )
while p.poll() is None:
    pass
# print(output.read())
# print(output.fn)
cmd = f"cat {output.fn}"
print(cmd)
os.system(cmd)

cat /tmp/tmp26nebruq
abcde
abcde
abcde
abcde
abcde


0